In [1]:
# from torch.utils.data import DataLoader
# import torch

# # Load the Widar_Dataset
# dataset = Widar_Dataset(root_dir='/Users/avilochab/Desktop/Grad_School/WiFi-CSI-Sensing-Benchmark-main/Benchmark/Data/Widardata')
# dataloader = DataLoader(dataset, batch_size=16, shuffle=True)  # Adjust batch size as needed


NameError: name 'Widar_Dataset' is not defined

In [2]:
import numpy as np
import torch
import torch.nn as nn
import argparse
from util import load_data_n_model
from torch.utils.data import DataLoader
from widar_model import RecognitionModel, StateMachineModel  # Make sure models are in widar_model.py

def train(recognition_model, state_machine_model, dataloader, num_epochs, learning_rate, criterion, device, sequence_length):
    recognition_model.to(device)
    state_machine_model.to(device)
    optimizer = torch.optim.Adam(
        list(recognition_model.parameters()) + list(state_machine_model.parameters()), lr=learning_rate
    )
    
    for epoch in range(num_epochs):
        recognition_model.train()
        state_machine_model.train()
        epoch_loss = 0
        epoch_accuracy = 0
        
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.type(torch.LongTensor).to(device)
            
            optimizer.zero_grad()
            
            # Collect features from RecognitionModel across a sequence
            sequence_features = []
            for t in range(sequence_length):
                output = recognition_model(inputs)  # Each time step gives a feature vector
                sequence_features.append(output.unsqueeze(1))  # Add time dimension
            
            # Concatenate features along the time dimension
            sequence_features = torch.cat(sequence_features, dim=1)  # Shape: (batch_size, sequence_length, feature_dim)
            
            # Pass the sequence of features to the StateMachineModel
            outputs = state_machine_model(sequence_features)
            
            # Compute loss and accuracy
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item() * inputs.size(0)
            predict_y = torch.argmax(outputs, dim=1)
            epoch_accuracy += (predict_y == labels).sum().item() / labels.size(0)
        
        epoch_loss = epoch_loss / len(dataloader.dataset)
        epoch_accuracy = epoch_accuracy / len(dataloader)
        print(f'Epoch:{epoch+1}, Accuracy:{epoch_accuracy:.4f}, Loss:{epoch_loss:.9f}')
    return


def test(recognition_model, state_machine_model, dataloader, criterion, device, sequence_length):
    recognition_model.eval()
    state_machine_model.eval()
    test_acc = 0
    test_loss = 0
    
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.type(torch.LongTensor).to(device)
            
            # Collect features from RecognitionModel across a sequence
            sequence_features = []
            for t in range(sequence_length):
                output = recognition_model(inputs)
                sequence_features.append(output.unsqueeze(1))
                
            sequence_features = torch.cat(sequence_features, dim=1)
            outputs = state_machine_model(sequence_features)
            
            # Calculate loss and accuracy
            loss = criterion(outputs, labels)
            predict_y = torch.argmax(outputs, dim=1)
            accuracy = (predict_y == labels).sum().item() / labels.size(0)
            test_acc += accuracy
            test_loss += loss.item() * inputs.size(0)
    
    test_acc = test_acc / len(dataloader)
    test_loss = test_loss / len(dataloader.dataset)
    print(f"Validation accuracy: {test_acc:.4f}, Loss: {test_loss:.5f}")
    return

    
def main():
    root = '/Users/avilochab/Desktop/Grad_School/WiFi-CSI-Sensing-Benchmark-main/Benchmark/Data/' 
    parser = argparse.ArgumentParser('WiFi Imaging Benchmark')
    parser.add_argument('--dataset', choices=['Widar'], default='Widar')
    parser.add_argument('--sequence_length', type=int, default=10)  # Sequence length for temporal data
    
    # Add the following line to ignore unrecognized arguments
    args, _ = parser.parse_known_args()
    
    # Load dataset and define DataLoader
    dataset = Widar_Dataset(root_dir=root)  # Assuming this returns the Widar dataset
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)  # Adjust batch size if needed
    
    # Initialize models
    recognition_model = RecognitionModel()
    state_machine_model = StateMachineModel(sequence_length=args.sequence_length, feature_dim=256, num_states=6)
    
    # Define criterion and device
    criterion = nn.CrossEntropyLoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Train and test
    train(
        recognition_model=recognition_model,
        state_machine_model=state_machine_model,
        dataloader=dataloader,
        num_epochs=10,  # Define epochs
        learning_rate=1e-3,
        criterion=criterion,
        device=device,
        sequence_length=args.sequence_length
    )
    test(
        recognition_model=recognition_model,
        state_machine_model=state_machine_model,
        dataloader=dataloader,
        criterion=criterion,
        device=device,
        sequence_length=args.sequence_length
    )

if __name__ == "__main__":
    main()



NameError: name 'Widar_Dataset' is not defined